# Data Exploration

5 MRI metrics: Cortical thickness (CT), mean diffusivity (MD), fractional anisotropy (FA). Moreover, functional connectivy data used to classify non-psychotic siblings.

Three different locations: Dublin, Maastricht, Cobre.

In [1]:
import pandas as pd

In [5]:
from os import chdir
chdir('/Users/mello/Desktop/Dataset')

In [10]:
FA_Maastricht = pd.read_csv('PARC_500.aparc_FA_cortexAv_mean_Maastricht.csv', delimiter = ',')
FA_Dublin = pd.read_csv('PARC_500.aparc_FA_cortexAv_mean_Dublin.csv', delimiter = ',')

MD_Maastricht = pd.read_csv('PARC_500.aparc_MD_cortexAv_mean_Maastricht.csv', delimiter = ',')
MD_Dublin = pd.read_csv('PARC_500.aparc_MD_cortexAv_mean_Dublin.csv', delimiter = ',')

CT_Maastricht = pd.read_csv('PARC_500.aparc_thickness_Maast.csv', delimiter = ',')
CT_Dublin = pd.read_csv('PARC_500.aparc_thickness_Dublin.csv', delimiter = ',')
CT_Cobre = pd.read_csv('PARC_500.aparc_thickness_Cobre.csv', delimiter = ',')


In [11]:
FA_Maastricht.shape

(124, 312)

In [12]:
FA_Dublin.shape

(115, 312)

In [13]:
FA_Maastricht.columns

Index(['Subject ID', 'Age', 'Sex', 'Group', 'lh_bankssts_part1_thickness',
       'lh_bankssts_part2_thickness',
       'lh_caudalanteriorcingulate_part1_thickness',
       'lh_caudalmiddlefrontal_part1_thickness',
       'lh_caudalmiddlefrontal_part2_thickness',
       'lh_caudalmiddlefrontal_part3_thickness',
       ...
       'rh_supramarginal_part5_thickness', 'rh_supramarginal_part6_thickness',
       'rh_supramarginal_part7_thickness', 'rh_frontalpole_part1_thickness',
       'rh_temporalpole_part1_thickness',
       'rh_transversetemporal_part1_thickness', 'rh_insula_part1_thickness',
       'rh_insula_part2_thickness', 'rh_insula_part3_thickness',
       'rh_insula_part4_thickness'],
      dtype='object', length=312)

In [16]:
MD_Maastricht.columns

Index(['Subject ID', 'Age', 'Sex', 'Group', 'lh_bankssts_part1_thickness',
       'lh_bankssts_part2_thickness',
       'lh_caudalanteriorcingulate_part1_thickness',
       'lh_caudalmiddlefrontal_part1_thickness',
       'lh_caudalmiddlefrontal_part2_thickness',
       'lh_caudalmiddlefrontal_part3_thickness',
       ...
       'rh_supramarginal_part5_thickness', 'rh_supramarginal_part6_thickness',
       'rh_supramarginal_part7_thickness', 'rh_frontalpole_part1_thickness',
       'rh_temporalpole_part1_thickness',
       'rh_transversetemporal_part1_thickness', 'rh_insula_part1_thickness',
       'rh_insula_part2_thickness', 'rh_insula_part3_thickness',
       'rh_insula_part4_thickness'],
      dtype='object', length=312)

In [18]:
CT_Maastricht.columns

Index(['Subject ID', 'Age', 'Sex', 'Group', 'lh_bankssts_part1_thickness',
       'lh_bankssts_part2_thickness',
       'lh_caudalanteriorcingulate_part1_thickness',
       'lh_caudalmiddlefrontal_part1_thickness',
       'lh_caudalmiddlefrontal_part2_thickness',
       'lh_caudalmiddlefrontal_part3_thickness',
       ...
       'rh_supramarginal_part5_thickness', 'rh_supramarginal_part6_thickness',
       'rh_supramarginal_part7_thickness', 'rh_frontalpole_part1_thickness',
       'rh_temporalpole_part1_thickness',
       'rh_transversetemporal_part1_thickness', 'rh_insula_part1_thickness',
       'rh_insula_part2_thickness', 'rh_insula_part3_thickness',
       'rh_insula_part4_thickness'],
      dtype='object', length=312)

In [25]:
FA_Maastricht.head(n=10)

,Subject ID,Age,Sex,Group,lh_bankssts_part1_thickness,lh_bankssts_part2_thickness,lh_caudalanteriorcingulate_part1_thickness,lh_caudalmiddlefrontal_part1_thickness,lh_caudalmiddlefrontal_part2_thickness,lh_caudalmiddlefrontal_part3_thickness,...,rh_supramarginal_part5_thickness,rh_supramarginal_part6_thickness,rh_supramarginal_part7_thickness,rh_frontalpole_part1_thickness,rh_temporalpole_part1_thickness,rh_transversetemporal_part1_thickness,rh_insula_part1_thickness,rh_insula_part2_thickness,rh_insula_part3_thickness,rh_insula_part4_thickness
0,con11,34,1,1,0.285,0.214,0.247,0.260,0.325,0.204,...,0.161,0.158,0.213,0.194,0.286,0.292,0.189,0.320,0.209,0.215
1,con12,26,2,1,0.245,0.255,0.293,0.282,0.238,0.246,...,0.174,0.195,0.239,0.307,0.261,0.212,0.213,0.190,0.196,0.257
2,con18,27,2,1,0.342,0.261,0.334,0.153,0.262,0.279,...,0.204,0.189,0.192,0.191,0.366,0.339,0.250,0.206,0.245,0.275
3,con19,36,1,1,0.365,0.266,0.261,0.244,0.186,0.266,...,0.153,0.182,0.246,0.357,0.282,0.401,0.336,0.308,0.219,0.286
4,con20,23,1,1,0.286,0.329,0.252,0.185,0.212,0.308,...,0.193,0.208,0.214,0.217,0.316,0.212,0.260,0.222,0.204,0.252
5,con21,44,2,1,0.357,0.402,0.394,0.276,0.227,0.145,...,0.193,0.185,0.212,0.396,0.285,0.411,0.298,0.334,0.284,0.356
6,con22,20,2,1,0.288,0.270,0.228,0.162,0.261,0.271,...,0.123,0.186,0.219,0.214,0.270,0.265,0.225,0.155,0.191,0.203
7,con23,25,2,1,0.248,0.161,0.307,0.207,0.268,0.247,...,0.170,0.204,0.248,0.230,0.247,0.191,0.228,0.138,0.165,0.182
8,con25,19,2,1,0.209,0.140,0.150,0.217,0.244,0.334,...,0.151,0.201,0.281,0.387,0.299,0.297,0.200,0.301,0.217,0.212
9,con27,43,2,1,0.203,0.223,0.214,0.143,0.303,0.275,...,0.194,0.271,0.265,0.433,0.243,0.200,0.152,0.132,0.134,0.141


## Reading the .mat files

In [28]:
import scipy.io

In [29]:
DTI_Dublin = scipy.io.loadmat('DTI_Dublin.mat')

In [30]:
DTI_Dublin

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Oct 23 09:47:58 2018',
 '__version__': '1.0',
 '__globals__': [],
 'nostreamlines_new': array([[array([[   0,  343,    0, ...,    0,    0,    0],
                [ 343,    0,    0, ...,    0,    0,    0],
                [   0,    0,    0, ...,    0,    0,    0],
                ...,
                [   0,    0,    0, ...,    0, 1653, 1060],
                [   0,    0,    0, ..., 1653,    0, 1992],
                [   0,    0,    0, ..., 1060, 1992,    0]], dtype=uint16),
         array([[   0,  777,    0, ...,    0,    0,    0],
                [ 777,    0,    0, ...,    0,    0,    0],
                [   0,    0,    0, ...,    0,    0,    0],
                ...,
                [   0,    0,    0, ...,    0,  365,  203],
                [   0,    0,    0, ...,  365,    0, 1312],
                [   0,    0,    0, ...,  203, 1312,    0]], dtype=uint16),
         array([[   0,  317,    0, ...,    0,    0,    0],

In [31]:
! pip install pymatreader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 KB 5.4 MB/s eta 0:00:00 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=d9bf8865637d950f7a9a00a963c613dcbe92c0b359daafaa360ce259f02f4dad
  Stored in directory: /Users/mello/Library/Caches/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future


In [32]:
from pymatreader import read_mat

In [34]:
DTI_Maastricht = read_mat('DTI_Maastricht.mat')

In [36]:
DTI_Maastricht.keys()

dict_keys(['__header__', '__version__', '__globals__', 'nostreamlines_new'])

In [37]:
df_DTI_M = pd.DataFrame(DTI_Maastricht['nostreamlines_new'])

ValueError: Must pass 2-d input. shape=(123, 308, 308)